# Auckland Hotel Advisory

## Section 1: Business Problem

### Situation

I have partnered with a hotel consultant to provide advisory services to assist hotel managers in improving profitability. We are based in Auckland, New Zealand and intend to refine our offer in the Auckland market before moving on to other metropolitan areas.  

Hotel profitability is largely a reflection of how well the hotel satisfies customers. 
Satisfaction with hotels is determined by a combination of: 

1. The on-site facilities and service levels
2. Easy connection to services that customers desire, but not provided directly by the hotel

#### Requirements for on-site facilities 
There is a great deal of variation in the requirements for on-site facilities, to meet varying customer needs. High end hotels require a broader range of facilities, better quality and greater service levels. However, mid tier and low end hotels can be hihgly satisfactory to customers and profitable with narrower and lower quality services.

Hotel industry experience is highly valuable in observing the service offering at a hotel and identifying improvments to meet the target market.

Increasingly it is possible, and desirable to use customer provided data as fact base to identify required improvements. Customer reviews are available on a variety of websites and can be analysed as a means of supporting the recommendations to hotel management. However, this is out of scope of current project.

#### Requirements for easy connection to services the customers desire
Easy connection to additional services is not well understood. In general terms, it is well known that customers require access to transport links, entertainment or business locations. We can expect there to be a wide variety of need for amenities or services that accompany a hotel visit because people travel for a variety of reasons and have a great variety of preferences.

Each hotel needs to understand its target market and align its service offerings in a number of ways:

**1. Location:** Location is typically fixed, although sometimes a major hotel group will decide its location is no longer appropriate to its needs and move. However, more often there may be opportunities to minimse the impact of a poor location for a target audience by improving transport linkages (e.g. Taxi and shuttle services).

**2. On-site offer:** Services widely used by customers could potentially be offered in house. Varying levels of restaurant, retail (e.g. gifts, convenience), personal services (e.g. laundry) and business support make up the mix of services that are sometimes improved to increase customer satisfaction. 

**3. Partnering and linkages:** Other services customers need are not offered on-site but effort by the hotel to assist with these contribute to customer satisfaction. This might, for example, include booking services, transport, recommendations for nearby attractions.

### Complication

With requirements for on-site facilities relatively well known, hotels need to focus on identifying the right connections to other services to integrate into the overall service offering  

### Business problem
What additional services do hotels in Auckland need to integrate with to improve their overall service offering to meet the needs of their specific target market?
* What is the target market for each hotel? How does this vary?
* What customer needs are associated with hotel visits for each target market?

## Section 2: Data

To answer the business problem we need:

1. List of hotels and locations
2. Identification of other services and facilities hotel customers are using associated with their hotel visit

### List of Hotels and Locations
FourSquare is a global location based data provider. The FourSquare API allows us to capture a list of hotels and specific locations within a specified area (endpoint is "get venue search"). However it will not return more that 50 in any specific location. To overcome this limitation, I will query the data at a suburb level, allowing up to 50 hotels per suburb.

A list of Auckland suburbs will be scraped from the internet. Auckland is located at coordinates of -36.86667 174.76667 and an appropriate list of suburbs can be found at https://geographic.org/streetview/new_zealand/auckland/auckland/index.html

There is no official recognition of suburb names, which may be problematic when using the Foursquare API. As a fall-back option, it would be possible to iterate over cooridnates within the Auckland region. The intended approach does not rely on a suburb for profiling or segmentation - it is the hotels themselves that will be analysed.

### Other services

The FourSqaure API allows us to identify venues that people check into after checking in to a speciic location. (endpoint is "get venues nextvenues"). The categories of the next venues visited will tell us the types of venues associated with each hotel. It will, for example, tell us if people a regularly visiting tourist attractions, the types of restaurants they are visiting and transport hubs.

### Analysis
The FourSquare API returns data in JSON format, which will be parsed into a Python Pandas dataframe for analysis.

The key parts of the analysis will be:

1. Scrape a list of suburbs in Auckland from the internet
2. Parse the list of suburbs into a Python Pandas dataframe.
3. Create an API call to FourSquare, using the suburb list to get all hotels in each suburb
4. Parse the list of hotels in JSON into a Python Pandas dataframe
5. Create an API call to FourSquare, using the hotel list to get venues checked into after the hotel
6. Parse the list of venues in JSON intop a Python Pandas datframe, including the categories
7. Merge the Hotel and Venues dataframes
8. Conduct a cluster analysis on the Hotels, using the venue categories as the descriptors
9. Profile clusters to understand the needs of customers at each hotel cluster.

### Outcome of Analysis
Having completed the data analysis we will be able to advise owners of hotels in any cluster what types of services are most likely to be valued by customers and therefore should be prioritised for development/partnering arrangements.